In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from keras.callbacks import CSVLogger
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
!rm -rf ./logs/GRU

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
def labels(filename):
    with open(filename, 'rb') as f:
        labels = np.load(f)
    return labels

In [5]:
training_data_y = labels("../Labels-Train.npy")
test_y = labels("../Labels-Test.npy")

training_data_x = pickle.load(open("TS-Train.pkl", "rb"))
test_x = pickle.load(open("TS-Test.pkl", "rb"))

print(training_data_x.columns[training_data_x.isna().any()].tolist())
print(test_x.columns[test_x.isna().any()].tolist())

training_data_x = training_data_x.dropna(axis=1)
test_x = test_x.dropna(axis=1)

print(training_data_x.shape, test_x.shape)

training_data_x = preprocessing.normalize(training_data_x,norm='max', axis=0)
test_x = preprocessing.normalize(test_x,norm='max', axis=0)

training_data_x = convert_3D(training_data_x)
test_x = convert_3D(test_x)

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

[]
[]
(12589, 769) (1574, 769)


In [6]:
GRUmodel = Sequential([
    GRU(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    GRU(64),
    #Dropout(0.3),
    #GRU(64),
    Dense(1, activation='sigmoid')
])

GRUmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy"]
)

log_dir = "logs/GRU/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

csv_logger = CSVLogger('keras-models/KerasGRU/training.csv',separator=",", append=False)

GRUmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = GRUmodel
model.save('keras-models/KerasGRU', save_format='h5')

Epoch 1/10
276/276 [==============================] - 25s 66ms/step - loss: 0.6916 - accuracy: 0.6043 - val_loss: 0.6876 - val_accuracy: 0.7660
Epoch 2/10
276/276 [==============================] - 17s 60ms/step - loss: 0.6834 - accuracy: 0.7728 - val_loss: 0.6797 - val_accuracy: 0.7660
Epoch 3/10
276/276 [==============================] - 17s 62ms/step - loss: 0.6755 - accuracy: 0.7728 - val_loss: 0.6722 - val_accuracy: 0.7660
Epoch 4/10
276/276 [==============================] - 17s 60ms/step - loss: 0.6679 - accuracy: 0.7728 - val_loss: 0.6649 - val_accuracy: 0.7660
Epoch 5/10
276/276 [==============================] - 22s 80ms/step - loss: 0.6605 - accuracy: 0.7728 - val_loss: 0.6579 - val_accuracy: 0.7660
Epoch 6/10
276/276 [==============================] - 22s 79ms/step - loss: 0.6534 - accuracy: 0.7728 - val_loss: 0.6511 - val_accuracy: 0.7660
Epoch 7/10
276/276 [==============================] - 20s 71ms/step - loss: 0.6465 - accuracy: 0.7728 - val_loss: 0.6445 - val_accuracy:

OSError: Unable to create file (unable to open file: name = 'keras-models/KerasGRU', errno = 21, error message = 'Is a directory', flags = 13, o_flags = 242)

In [ ]:
%tensorboard --logdir logs/GRU/fit